In [1]:
import os
import time
import numpy as np
import datetime
import pandas as pd
#import stldecompose as stl
import glob
import matplotlib.pyplot as plt
#import cntk as C


In [2]:
# Funcion que hace ENCODE de valores ciclicos 
# agregando 2 columnas con SIN y COS
def encode(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data



def printx(texto):
    print(datetime.datetime.now().strftime("<%Y-%m-%d %H:%M:%S> " + texto))   
    return


In [3]:
# ********************************************************************************
#                                    M A I N 
# ********************************************************************************

printx("Iniciando ...")



# Training parameters
BATCH_SIZE = 200
EPOCHS = 500
lrngEpochSize = 200
#learning_rate = [(1, 0.3), (5, 0.1), (2, 0.08), (5, 0.04),(5, 0.01),(2, 0.005),(2, 0.001)]
#learning_rate = [(10, 0.1), (5, 0.05), (5, 0.01), (2, 0.005), (2, 0.001)]
learning_rate = 0.7
momentumRate = 0.9
l2Reglr = 0 #0.001


dropPercent = 0.2
seasonLength = 31
inputCount = 35
outputCount = 1
val_size=0.10       # 0.20 
test_size=0.10      # 0.25
atmToReview = 20
prepareData = False
plotValues = False
plotComponents = False
displayCount = 20






<2018-09-17 17:52:44> Iniciando ...


In [6]:
# Lee TODOS los archivos CSV
def GetDfFromFiles(path):
    all_files = glob.glob(os.path.join(path, "*.csv"))
    numFiles=0
    df_from_each_file = (
        pd.read_csv(
            source_file_name
            ,low_memory=False
            ,index_col=(0)
            ,sep=';'
            ,names=['IDLOG','FECHA','HORA','IDOBJETO','HOPPER','DISPENSADO','REMANENTE','MONEDA','DENOMINACION','IDTOM']
            ,header=None    
        ) for source_file_name in all_files
    )
    df = pd.concat(df_from_each_file, ignore_index=True) 
    # print(df.dtypes)
    # {
    # 'FECHA': np.int64,
    # 'HORA': np.object,
    # 'IDOBJETO': np.object,
    # 'HOPPER': np.object,
    # 'DISPENSADO': np.int64,
    # 'REMANENTE': np.int64,
    # 'MONEDA': np.int64,
    # 'DENOMINACION': np.int64,
    # 'IDTOM': np.int64,
    # 'ImporteDisp: np.int64 
    # }
    return df


In [13]:
# Lee TODOS los archivos CSV y prepara el Dataframe
path = "c:\\Users\\Mariano\\OneDrive\\Proyectos\\SysAssap\\Predictivo\\SRC\\PredCashML\\PredCashML\\Data\\"          
df  = GetDfFromFiles(path)

# Agrega columna ImporteDisp multiplicando la denominacion del billete respecto de su cantidad
df['ImporteDisp'] = df.DISPENSADO * df.DENOMINACION


In [34]:
df2 = df[(df['DENOMINACION'] > 0)] \
    .groupby(['FECHA', 'HORA','IDOBJETO', 'MONEDA' ])\
    .agg({'ImporteDisp': np.sum })\
    .reset_index()\
    .sort_values(['IDOBJETO', 'MONEDA', 'FECHA', 'HORA'])

# Genera un dataframe solo con la columna fecha conteniendo todas las fechas unicas
df3 = pd.DataFrame({'FECHA': df2.FECHA.unique()})
df3.set_index('FECHA')

# Obtiene la lista de ATMs
atms = df.IDOBJETO.unique() 
numAtm =  len(atms)


In [41]:
setatmName = atms[:1][0]
print(atmName)

dft = df2.loc[(df2.IDOBJETO == atmName) & (df2.MONEDA == 32) & (df2.FECHA >= 20170101) & (df2.ImporteDisp >= 0),['FECHA','ImporteDisp']]
    
dft.reset_index(drop=True)

dft.set_index('FECHA') 
dft



S1ERI769


,FECHA,ImporteDisp
0,20170101,175900
319,20170101,175900
638,20170101,175900
957,20170101,175900
1276,20170101,175900
1595,20170101,175900
1914,20170101,175900
2233,20170101,175900
2552,20170101,175900
2871,20170101,175900


In [8]:
def reformatDataframe(df, numAtm):
    

    # Muestra el primer nombre de ATM que aparece
    #df.IDOBJETO.unique()[0]


    
    if numAtm <= 0:
        
    
    
    for idObj in atms[:numAtm]:            
        print("Procesando ATM:",idObj)
        #Seleciona 'FECHA','ImporteDisp' de un ATM solo para moneda 32 (pesos) a partir del año 2017 Con importe >= 0
        dft = df2.loc[(df2.IDOBJETO == idObj) & (df2.MONEDA == 32) & (df2.FECHA >= 20170101) & (df2.ImporteDisp >= 0), ['FECHA','ImporteDisp']].reset_index(drop=True)       
        # Le cambia el nombre a la columna ImporteDisp por el nombre del ATM    
        dft.columns = ['FECHA',idObj]       
        dft[idObj + '_ORI'] = dft[idObj]        
        # ------------------------------------------------------------------------------------------------------------------------
        dft[idObj] = dft[idObj].apply(lambda x: np.log(x+1000))                        #       Aplica Logaritmo 
        # ------------------------------------------------------------------------------------------------------------------------
        # Indexa por la fecha para ayudar en el JOIN que se va a hacer con df3
        dft.set_index('FECHA')     
        # Hace el join por la columna fecha
        df3 = pd.merge(df3, dft, on='FECHA', how='outer')   


    # Convierte columna Fecha(INT64) a la columna DIA(datetime)
    df3['DIA'] = df3['FECHA'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))

    # Agrega el numero de dia de la semana
    df3['DIASEMANA'] = df3['DIA'].dt.weekday + 1
    
    # Agraga las columnas ciclicas de dia de la semana
    df3 = pcutl.encode(df3,'DIASEMANA',7)
    
    #Elimina la columna temporal de DIASEMANA
    df3.drop(['DIASEMANA'], axis=1 , inplace=True)
    
    # Asigna DIA como indice del dataframe
    df3.set_index('DIA', inplace=True)
    
    # Elimina la columna fecha
    df3.drop(['FECHA'], axis=1 , inplace=True)

    #Se asegura que tenga todas las fechas
    fedDesde = df3.index.min()
    fedHasta = df3.index.max()
    dateRange = pd.date_range(start=fedDesde,end=fedHasta)

    #Genera el dataframe con todas las fechas
    fechasTodas = pd.DataFrame({'DIA': dateRange})
    fechasTodas.set_index('DIA', inplace=True)
    df3 = pd.merge(df3, fechasTodas, left_index=True, right_index=True , how='outer')   
    
    return df3, atms[0:numAtm-1]




In [7]:
printx("Preparando datos ...")


printx("Leyendo Archivos ...")
 


<2018-09-17 17:55:29> Preparando datos ...
<2018-09-17 17:55:29> Leyendo Archivos ...


In [12]:
df.head(10)

,FECHA,HORA,IDOBJETO,HOPPER,DISPENSADO,REMANENTE,MONEDA,DENOMINACION,IDTOM
0,20170101,00:00:01.515,S1ERI769,A,0,100,32,10,2
1,20170101,00:00:01.515,S1ERI769,B,6,994,32,50,2
2,20170101,00:00:01.515,S1ERI769,C,0,0,32,0,2
3,20170101,00:00:01.515,S1ERI769,D,1756,2244,32,100,2
4,20170101,00:00:01.516,S1ERI771,A,0,0,32,10,2
5,20170101,00:00:01.516,S1ERI771,B,0,0,32,50,2
6,20170101,00:00:01.516,S1ERI771,C,0,0,32,0,2
7,20170101,00:00:01.516,S1ERI771,D,2838,4162,32,100,2
8,20170101,00:00:01.516,S1ERI784,A,0,0,32,10,2
9,20170101,00:00:01.516,S1ERI784,B,0,0,32,50,2


In [ ]:

printx("Procesando datos ...")
dfSeries,atms = reformatDataframe(df,atmToReview) # Lee los primeros 10 atm solamente
printx("Guardando  datos ...")
dfSeries.to_json("df3.json")
np.save("atms.npy",atms)
printx("Proceso Completo.")


In [ ]:
printx("Cargando Datos ...")
dfSeries=pd.read_json("df3.json")
atms=np.load("atms.npy")
#atmToReview = len(atms)


In [ ]:
# ---------------------------------------------------------------
#   Genera los graficos
# ---------------------------------------------------------------
if plotValues:
    printx("Generando graficos de Series ...")
    # Genera los graficos de las series
    pcplt.plotSeriesValues(dfSeries,atms)    
    printx("Generando graficos densidades expectrales ...")
    #Genera Graficos de las densidades espectrales
    pcplt.plotSeriesPsd(dfSeries,atms)    

if plotComponents:
    printx("Generando graficos Componentes ...")
    #Genera Graficos de las las componentes
    pcplt.plotSeriesComponentes(dfSeries,atms,seasonLength)    


In [ ]:


def prepareDatasets(deseason, validationPrct, testPrct, outputCount, inputCount):    
    serieCount = deseason.size

    # parto el dataset en  train, validatation y test
    validation_size = int(serieCount * validationPrct)
    test_size = int(serieCount * testPrct)
    result_x = {"todo": [], "train": [], "val": [], "test": []}
    result_y = {"todo": [], "train": [], "val": [], "test": []}
    next_val = 0
    next_test = 0
    current_set = "train"
    max_i=0
    for i in range(0,serieCount-outputCount-inputCount+1):
        if i >= next_val:
            current_set = "val"
            next_val = i + int(serieCount / validation_size)
        elif i >= next_test:
            current_set = "test"
            next_test = i + int(serieCount / test_size)
        else:
            current_set = "train"
        result_x[current_set].append(deseason[i:i+inputCount])    
        result_x["todo"].append(deseason[i:i+inputCount])    
        result_y[current_set].append(deseason[i+inputCount:i+inputCount+outputCount])
        result_y["todo"].append(deseason[i+inputCount:i+inputCount+outputCount])
        max_i = i

    # hago el reshape necesario por la red neuronal para tomarlo como secuencia
    for current_set in ["train", "val", "test", "todo"]:
        result_x[current_set] = np.array(result_x[current_set])    
        result_x[current_set] = result_x[current_set].reshape(result_x[current_set].shape + (1,))
        result_y[current_set] = np.array(result_y[current_set])
    return result_x, result_y





